Transformer(Bert)

In [1]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.1
    Uninstalling huggingface-hub-0.19.1:
      Successfully uninstalled huggingface-hub-0.19.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 12.7 MB/s eta 0

In [2]:
import random
import pyarrow as pa
import datasets
import tqdm
import string

In [3]:
from datasets import load_dataset
dts=load_dataset('SeyedAli/Persian-Text-Sentiment')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/55852 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/13964 [00:00<?, ? examples/s]

In [4]:
dts

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 55852
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 13964
    })
})

In [5]:
!pip install transformers[torch]

In [6]:
import torch
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification,TrainingArguments,Trainer

In [7]:
def gpu_check():
  torch.__version__
  #If ther 's a GPU available...
  if torch.cuda.is_available():
    #Tell Pytorch to use GPU.
    device = torch.device("cuda")
    print("There are %d GPU(s) avalilable." % torch.cuda.device_count())
    print('We will use the GPU:',torch.cuda.get_device_name(0))
    #If not..
  else:
    print('No GPU available,using the CPU instead.')
    device = torch.device("cpu")
    return device

In [8]:
device=gpu_check()
print(device)

There are 1 GPU(s) avalilable.
We will use the GPU: Tesla T4
None


In [9]:
label2id={'negetive':0,'positive':1}
id2label={0:'negetive',1:'positive'}

In [10]:
modelname="HooshvareLab/bert-base-parsbert-uncased"
tokenizer=AutoTokenizer.from_pretrained(modelname,model_max_length=100, add_special_tokens = True, add_prefix_space=True)
model=AutoModelForSequenceClassification.from_pretrained(
    modelname,num_labels=2,id2label=id2label,label2id=label2id
).to(device)
data_collector = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from sklearn.metrics import precision_recall_fscore_support,accuracy_score

In [12]:
def preprocess_function(examples):
  return tokenizer(examples["text"],truncation=True)
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, fscore, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy=accuracy_score(labels, preds)
    metrics = {
        'precision': precision,
        'recall': recall,
        'F1-score': fscore,
        'accuracy': accuracy,
    }
    return metrics

In [13]:
tokenized_data=dts.map(preprocess_function,batched=True)

Map:   0%|          | 0/55852 [00:00<?, ? examples/s]

Map:   0%|          | 0/13964 [00:00<?, ? examples/s]

Injecting LoRA to the BERT model

In [14]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=4, lora_alpha=1, lora_dropout=0.1
)

In [15]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

In [16]:
training_args = TrainingArguments(
    output_dir="Persian-Text-Sentiment-Bert-LORA/",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy ="epoch",
    save_strategy ="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)

In [17]:
trainer =Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    data_collator=data_collector,
    compute_metrics=compute_metrics
)

In [18]:
print("شروع آموزش...")
trainer.train()

شروع آموزش...


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
1,0.393600,0.386482,0.843127,0.838943,0.838451,0.838943
2,0.373200,0.370097,0.850630,0.845603,0.845048,0.845603
3,0.357400,0.360037,0.852351,0.848253,0.847810,0.848253
4,0.361800,0.355032,0.852821,0.849828,0.849509,0.849828
5,0.350900,0.350338,0.855086,0.851404,0.851017,0.851404
6,0.339900,0.348835,0.855257,0.852263,0.851951,0.852263
7,0.347800,0.346615,0.856677,0.853194,0.852835,0.853194
8,0.347100,0.344090,0.857548,0.854197,0.853854,0.854197
9,0.341200,0.344162,0.856855,0.853695,0.853371,0.853695
10,0.335100,0.343837,0.857506,0.854125,0.853779,0.854125


TrainOutput(global_step=34910, training_loss=0.36434650972967875, metrics={'train_runtime': 5182.4452, 'train_samples_per_second': 107.772, 'train_steps_per_second': 6.736, 'total_flos': 1.878185871049488e+16, 'train_loss': 0.36434650972967875, 'epoch': 10.0})

In [19]:
trainer.evaluate()

{'eval_loss': 0.34383702278137207,
 'eval_precision': 0.8575060332137552,
 'eval_recall': 0.8541248925809224,
 'eval_F1-score': 0.8537791689095465,
 'eval_accuracy': 0.8541248925809224,
 'eval_runtime': 56.7621,
 'eval_samples_per_second': 246.009,
 'eval_steps_per_second': 15.38,
 'epoch': 10.0}

In [20]:
trainer.save_model('sentiment')

In [21]:
!zip -r emotion_bert_model.zip 'sentiment'

  adding: sentiment/ (stored 0%)
  adding: sentiment/adapter_model.safetensors (deflated 8%)
  adding: sentiment/README.md (deflated 65%)
  adding: sentiment/adapter_config.json (deflated 46%)
  adding: sentiment/training_args.bin (deflated 50%)


In [ ]:
!pip install huggingface_hub

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [ ]:
trainer.push_to_hub('SeyedAli/Persian-Text-Emotion-Bert-LORA')

adapter_model.safetensors:   0%|          | 0.00/160k [00:00<?, ?B/s]

events.out.tfevents.1699697696.36d742872af9.1859.0:   0%|          | 0.00/5.79k [00:00<?, ?B/s]

events.out.tfevents.1699698648.36d742872af9.1859.1:   0%|          | 0.00/5.79k [00:00<?, ?B/s]

Upload 9 LFS files:   0%|          | 0/9 [00:00<?, ?it/s]

events.out.tfevents.1699699938.36d742872af9.1859.3:   0%|          | 0.00/4.41k [00:00<?, ?B/s]

...

events.out.tfevents.1699699972.36d742872af9.1859.4:   0%|          | 0.00/4.41k [00:00<?, ?B/s]

events.out.tfevents.1699699984.36d742872af9.1859.5:   0%|          | 0.00/19.0k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

events.out.tfevents.1699705059.36d742872af9.1859.6:   0%|          | 0.00/417 [00:00<?, ?B/s]

'https://huggingface.co/SeyedAli/Persian-Text-Sentiment-Bert-LORA/tree/main/'

In [22]:
from peft import PeftModel
from transformers import pipeline
modelname="SeyedAli/Persian-Text-Sentiment-Bert-LORA"
tokenizer=AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased",model_max_length=100, add_special_tokens = True)
model=AutoModelForSequenceClassification.from_pretrained("HooshvareLab/bert-base-parsbert-uncased",num_labels=2,id2label=id2label,label2id=label2id)
model = PeftModel.from_pretrained(model, modelname)
pipe = pipeline("text-classification", model=model,tokenizer=tokenizer)
pipe('خیلی کتاب خوبی بود')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The model 'PeftModelForSequenceClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GPT2ForSequenceClassification',

[{'label': 'positive', 'score': 0.9820583462715149}]